In [143]:
import pandas as pd

In [144]:
df = pd.read_csv('nba_games.csv', index_col=0)

In [145]:
df = df.sort_values("date")

In [146]:
df = df.reset_index(drop=True)

In [147]:
del df["mp.1"]
del df["mp_opp.1"]
del df["index_opp"]

In [148]:
def add_target(team):
    team["target"] = team["won"].shift(-1)
    return team

df = df.groupby("team", group_keys=False).apply(add_target)

C:\Users\tipaa\AppData\Local\Temp\ipykernel_6728\3228710534.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team["target"] = team["won"].shift(-1)
C:\Users\tipaa\AppData\Local\Temp\ipykernel_6728\3228710534.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team["target"] = team["won"].shift(-1)
C:\Users\tipaa\AppData\Local\Temp\ipykernel_6728\3228710534.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining al

In [149]:
df["target"][pd.isnull(df["target"])] = 2

C:\Users\tipaa\AppData\Local\Temp\ipykernel_6728\997784747.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["target"][pd.isnull(df["target"])] = 2
C:\Users\tipaa\AppData\Local\Temp\ipykernel_6728\997784747.py:1: SettingWithCopyWarning:

In [150]:
df["target"] = df["target"].astype(int, errors="ignore")

In [151]:
nulls = pd.isnull(df)

In [152]:
nulls = nulls.sum()

In [153]:
nulls = nulls[nulls > 0]

In [154]:
valid_columns = df.columns[~df.columns.isin(nulls.index)]

In [155]:
df = df[valid_columns].copy()

In [156]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

rr = RidgeClassifier(alpha=1)
split = TimeSeriesSplit(n_splits=3)

sfs = SequentialFeatureSelector(rr, n_features_to_select=30, direction="forward", cv=split)

In [157]:
removed_columns = ["season", "date", "won", "target", "team", "team_opp"]

In [158]:
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [159]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])

In [160]:
sfs.fit(df[selected_columns], df["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [161]:
predictors = list(selected_columns[sfs.get_support()])

In [164]:
def backtest(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    seasons = sorted(data["season"].unique())
    
    for i in range(start, len(seasons), step):
        season = seasons[i]
        
        train = data[data["season"] < season]
        test = data[data["season"] == season]
        
        model.fit(train[predictors], train["target"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        
        combined = pd.concat([test["target"], preds], axis=1)
        combined.columns = ["actual", "predicted"]
        
        all_predictions.append(combined)
        
    return pd.concat(all_predictions)

In [165]:
predictions = backtest(df, rr, predictors)

In [166]:
predictions

,actual,predicted
5236,1,0
5237,0,0
5238,1,1
5239,0,1
5240,0,1
...,...,...
17753,0,1
17754,1,1
17755,0,1
17756,2,0


In [170]:
from sklearn.metrics import accuracy_score

predictions = predictions[predictions["actual"] != 2]
accuracy_score(predictions["actual"], predictions["predicted"])

0.5429074607748959

In [171]:
df.groupby("home").apply(lambda x: x[x["won"] == 1].shape[0] / x.shape[0])

C:\Users\tipaa\AppData\Local\Temp\ipykernel_6728\1818935125.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby("home").apply(lambda x: x[x["won"] == 1].shape[0] / x.shape[0])


home
0.0    0.430341
1.0    0.569659
dtype: float64

In [172]:
df_rolling = df[list(selected_columns) + ["won", "team", "season"]]

In [173]:
def find_team_averages(team):
    rolling = team.rolling(10).mean()
    
    return rolling

df_rolling = df_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)

DataError: Cannot aggregate non-numeric type: object

In [174]:
rolling_cols = [f"{col}_10" for col in df_rolling.columns]
df_rolling.columns = rolling_cols

df = pd.concat([df, df_rolling], axis=1)

In [175]:
df = df.dropna()

In [176]:
def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))

df["home_next"] = add_col(df, "home")
df["team_opp_next"] = add_col(df, "team_opp")
df["date_next"] = add_col(df, "date")

C:\Users\tipaa\AppData\Local\Temp\ipykernel_6728\30860547.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))
C:\Users\tipaa\AppData\Local\Temp\ipykernel_6728\30860547.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))
C:\Users\tipaa\AppData

In [177]:
df = df.copy()

In [178]:
full = df.merge(df[rolling_cols + ["team_opp_next", "date_next", "team"]], left_on=["team", "date_next"], right_on=["team_opp_next", "date_next"])

In [179]:
removed_columns = list(full.columns[full.dtypes == "object"]) + removed_columns

In [180]:
selected_columns = full.columns[~full.columns.isin(removed_columns)]

In [181]:
sfs.fit(full[selected_columns], full["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [182]:
predictors = list(selected_columns[sfs.get_support()])

In [184]:
predictions = backtest(full, rr, predictors)

In [185]:
accuracy_score(predictions["actual"], predictions["predicted"])

0.5752481588216458